In [ ]:
from Board import *
from GameConstants import *
from collections import Counter
import copy
import random
from random import choice, randint
import numpy as np

In [ ]:
class Tile:
  """
  Class: Tile
  ---------------------------
  A Tile represents a single square on our square gameboard.  A tile
  has a resource type, a roll number, and, optionally, can have a road
  or settlement be built on it by a single player.
  ---------------------------
  """


  def __init__(self, resource, number, x, y):
    self.resource = resource
    self.x = x
    self.y = y
    self.number = number
    self.player = None
    self.structure = Structure["NONE"] # Settlement, vertical road, or horizontal road


  """
  Method: isOccupied
  ---------------------------
  Parameters: None
  Returns: True/False depending on whether or not the tile has been used

  Returns whether or not this tile is occupied
  ---------------------------
  """
  def isOccupied(self):
    return self.structure != Structure["NONE"]


  """
  Method: settle
  ---------------------------
  Parameters:
    playerIndex: the index of the player that is settling on this tile
  Returns: NA

  Marks this tile as settled by the given player.  Throws an exception if
  this tile has already been used.
  ---------------------------
  """
  def settle(self, playerIndex):
    if self.isOccupied():
      raise Exception("This tile is already used!")
    self.player = playerIndex
    self.structure = Structure["SETTLEMENT"]
  
  """
  Method: upgrade
  ---------------------------
  Parameters:
    playerIndex: the index of the player that is settling on this tile
  Returns: NA

  Marks this tile as upgraded to a city by the given player.  Throws an exception if
  this tile is not settled by the proper player or doesn't have a settlement yet.
  Note that the tile is still within the settlements array so that all the adjacency
  methods work fine.
  ---------------------------
  """
  def upgrade(self, playerIndex):
    if self.structure != Structure["SETTLEMENT"]:
      raise Exception("This tile is not settled yet!")
    if self.player != playerIndex:
      raise Exception(str(self.player)+" has already settled here!")
    self.structure = Structure["CITY"]


  """
  Method: buildRoad
  ---------------------------
  Parameters:
    playerIndex: the index of the player that is building a road on this tile
  Returns: NA

  Builds a road owned by the given player on this tile.
  ---------------------------
  """
  def buildRoad(self, playerIndex):
    if self.isOccupied():
      raise Exception("Tile " + str(self) + " is already used!")
    self.player = playerIndex
    self.structure = Structure["ROAD"]


  """
  Method: __repr__ method
  ---------------------------
  Parameters: NA
  Returns: NA

  Prints out a description of this tile
  ---------------------------
  """
  def __repr__(self):
    val = "--------------TILE INFO AT (" + str(self.x) + ", " + str(self.y) + ")---------------\n"
    val += "Owned by player: " + str(self.player) +"\n"
    val += "Structure: " + str(self.structure) +"\n"
    val += "Resource Type: " + str(self.resource) +"\n"
    val += "Tile number: " + str(self.number) +"\n"
    return val


  """
  Method: strRepresentation
  ---------------------------
  Parameters: NA
  Returns: a "stringified" version of this tile

  Returns a string representing this tile.  If this tile is unused, this method
  returns "-".  If it is a settlement, it returns "S#", where # = the player who owns
  the settlement.  IF it is a road, it returns "R#", where # = the player who owns
  the road.  (Note: This method assumes the player index will never be more than 1 digit,
  since all tile string representations are length 2).
  ---------------------------
  """
  def strRepresentation(self):
    if not self.isOccupied(): return "--"
    elif self.structure == Structure["ROAD"]: return "R" + str(self.player)
    elif self.structure == Structure["SETTLEMENT"]: return "S" + str(self.player)
    raise Exception("strRepresentation - invalid tile")
  


In [ ]:
def test_tile_class():
    # Create a tile object
    tile = Tile(2, 6, 2, 3)

    print(tile)

    # Test the initial state of the tile
    assert tile.resource == 2
    assert tile.x == 2
    assert tile.y == 3
    assert tile.number == 6
    assert tile.player is None

    # Test isOccupied method
    assert tile.isOccupied() == False

    # Test settle method
    tile.settle(1)
    assert tile.isOccupied() == True
    assert tile.structure == Structure["SETTLEMENT"]
    assert tile.player == 1


    print(tile.strRepresentation())

    print(tile.__repr__())

    print(tile.structure)

    # Test upgrade method
    tile.upgrade(1)
    assert tile.structure == Structure["CITY"]

    print(tile.structure)

    print("All tests passed!")

test_tile_class()

In [ ]:
class BasicBoard:
  """
  Class: BasicBoard
  ---------------------------
  A BasicBoard is an n x n grid of Tiles representing a simplified
  version of the Settlers of Catan gameboard.  Every Tile can be built
  on, and every tile has a resource type and roll number, along with an x and y.
  A BasicBoard contains an n x n grid of Tiles, as well as a list of all
  built settlements and a list of all built roads.
  ---------------------------
  """

  def __init__(self, size=5):
    self.board = []

    possibleResources = [] 
    for resource in ResourceTypes.values():
      possibleResources += 5*[resource]
      random.shuffle(possibleResources)

    possiblenumbers = [2,2,3,3,4,4,5,5,5,6,6,6,8,8,8,9,9,9,10,10,10,11,11,12,12]
    random.shuffle(possiblenumbers)
    for i in range(size):
      boardRow = []
      for j in range(size):
        boardRow.append(Tile(possibleResources.pop(), possiblenumbers.pop(), i, j))
      self.board.append(boardRow)

    self.settlements = []
    self.roads = []
    self.size = size


  """
  Method: getTile
  ---------------------------
  Parameters:
    x: the x coordinate of the tile to get
    y: the y coordinate of the tile to get
  Returns: the Tile object at that (x,y), or None if the coordinates are out of bounds

  Returns the tile on the board at the given coordinates, or None if the
  coordinates are out of bounds.
  ---------------------------
  """
  def getTile(self, x, y):
    if 0 <= x < self.size and 0 <= y < self.size:
      return self.board[y][x]
    return None


  """
  Method: printBoard
  ---------------------------
  Parameters: NA
  Returns: NA

  Prints out an ASCII representation of the current board state.
  It does this by printing out each row inside square brackets.
  Each tile is either '--' if it's unused, 'RX' if it's a road,
  or 'SX' if it's a settlement.  The 'X' in the road or settlement
  representation is the player who owns that road/settlement.
  ---------------------------
  """
  def printBoard(self):
    # Print top numbers
    s = "     "
    for i in range(self.size):
      s += str(i) + "   "
    print(s)

    for i, row in enumerate(self.board):
      s = str(i) + " ["
      for tile in row:
        s += " " + tile.strRepresentation() + " "
      print(s + "]")


  """
  Method: applyAction
  ---------------------------
  Parameters:
    playerIndex: the index of the player that is taking an action
    action: a tuple (ACTION_TYPE, Tile) representing the action to be
            taken and where that action should be taken.
  Returns: NA

  Updates the board to take the given action for the given player.  The action
  can either be building a settlement or building a road.
  ---------------------------
  """
  def applyAction(self, playerIndex, action):
    if action == None: return
    
    # Mark the tile as a settlement
    if action[0] == Actions["SETTLE"]:
      tile = action[1]
      tile.settle(playerIndex)
      self.settlements.append(tile)

    # Or mark the tile as a road
    elif action[0] == Actions["ROAD"]:
      tile = action[1]
      tile.buildRoad(playerIndex)
      self.roads.append(tile)

    # Or mark the tile as a city
    elif action[0] == Actions["CITY"]:
      tile = action[1]
      tile.upgrade(playerIndex)

  """
  Method: getNeighborTiles
  ---------------------------
  Parameters:
    tile: the Tile object to find the neighbors of
  Returns: a list of all the adjacent tiles to this tile

  Returns a list of all of the tiles immediately surrounding
  the passed-in tile
  ---------------------------
  """
  def getNeighborTiles(self, tile, diagonals=False):
    neighbors = []
    for dx in range(-1, 2):
      for dy in range(-1, 2):

        # Ignore the original tile
        if dx == 0 and dy == 0: continue

        # Optionally ignore diagonals
        if not diagonals and (dx != 0 and dy != 0): continue

        # If this location is in bounds, add the tile to our list
        currTile = self.getTile(tile.x + dx, tile.y + dy)
        if currTile != None:
          neighbors.append(currTile)

    return neighbors


  """
  Method: getUnoccupiedNeighbors
  ---------------------------
  Parameters:
    tile: the tile to return the neighbors for
  Returns: a list of all the unoccupied neighbors for this tile

  Returns a list of all of the unoccupied tiles adjacent to this tile
  ---------------------------
  """
  def getUnoccupiedNeighbors(self, tile, diagonals=True):
    neighbors = self.getNeighborTiles(tile, diagonals=diagonals)

    unoccupied = []
    for neighbor in neighbors:
      if not neighbor.isOccupied():
        unoccupied.append(neighbor)
    return unoccupied

  """
  Method: getResourcesFromDieRoll
  ---------------------------
  Parameters:
    tile: the tile to return the neighbors for
  Returns: a list of all the unoccupied neighbors for this tile

  Returns a list of all of the unoccupied tiles adjacent to this tile
  ---------------------------
  """
  def getResourcesFromDieRoll(self, playerIndex, dieRoll):
    resources = Counter() 
    
    for settlement in self.settlements:
      if settlement.number == dieRoll and settlement.player == playerIndex:
        resources[settlement.resource] += 1

        if settlement.structure == Structure["CITY"]:
            resources[settlement.resource] += 1
                
    return resources

  """
  Method: getOccupiedNeighbors
  ---------------------------
  Parameters:
    tile: the tile to return the neighbors for
  Returns: a list of all the occupied neighbors for this tile

  Returns a list of all of the occupied tiles adjacent to this tile
  ---------------------------
  """
  def getOccupiedNeighbors(self, tile, diagonals=True):
    neighbors = self.getNeighborTiles(tile, diagonals=diagonals)

    occupied = []
    for neighbor in neighbors:
      if neighbor.isOccupied():
        occupied.append(neighbor)

    return occupied


  """
  Method: isValidSettlementLocation
  ---------------------------
  Parameters:
    tile: the tile to check
  Returns: True/False depending on whether or not that tile is a valid settlement location

  Returns whether or not a settlement can validly be built on the given tile
  ---------------------------
  """
  def isValidSettlementLocation(self, tile):
    # It's a valid settlement location if there are no other settlements
    # within 1 space of this one
    occupiedNeighbors = self.getOccupiedNeighbors(tile)
    for neighbor in occupiedNeighbors:
      if neighbor.structure == Structure["SETTLEMENT"]:
        return False

    return True


  """
  Method: getUnoccupiedRoadEndpoints
  ---------------------------
  Parameters:
    tile: the road to get endpoints for
  Returns: a list of all the unoccupied endpoints of the given road 

  Returns all the unoccupied endpoints of the given road.  Note
  that this could be up to 3 endpoints (since roads have no direction)
  ---------------------------
  """
  def getUnoccupiedRoadEndpoints(self, tile):
    if not tile.isOccupied() or tile.structure != Structure["ROAD"]:
      raise Exception("getUnoccupiedRoadEndpoints - not a road!")

    return self.getUnoccupiedNeighbors(tile, diagonals=False)
  
  def deepcopy(self):
    return copy.deepcopy(self)


In [ ]:
def test_basic_board():
    # Create a BasicBoard object
    board = BasicBoard()

    # Test getTile method
    assert board.getTile(0, 0) is not None
    assert board.getTile(2, 2) is not None

    # Test printBoard method
    board.printBoard()

    print(board.getTile(0, 0))

    # Test applyAction method
    playerIndex = 1

    action = [Actions["SETTLE"], board.getTile(0, 0)]
    board.applyAction(playerIndex, action)
    assert len(board.settlements) == 1

    print(board.getTile(0, 0))

    board.printBoard()

    action = (Actions["ROAD"], board.getTile(0, 1))
    board.applyAction(playerIndex, action)
    assert len(board.roads) == 1

    action = (Actions["CITY"], board.getTile(0, 0))
    board.applyAction(playerIndex, action)

    # Test getNeighborTiles method
    tile = board.getTile(1, 1)
    neighbors = board.getNeighborTiles(tile)
    # print(neighbors)

    # Test getUnoccupiedNeighbors method
    unoccupied_neighbors = board.getUnoccupiedNeighbors(tile)
    # print(unoccupied_neighbors)

    # Test getResourcesFromDieRoll method
    dieRoll = 2
    resources = board.getResourcesFromDieRoll(playerIndex, dieRoll)
    print(resources)

    # Test getOccupiedNeighbors method
    occupied_neighbors = board.getOccupiedNeighbors(tile)
    #print(occupied_neighbors)

    # Test isValidSettlementLocation method
    print(board.isValidSettlementLocation(tile))

    # Test getUnoccupiedRoadEndpoints method
    road_endpoints = board.getUnoccupiedRoadEndpoints(board.getTile(0, 1))
    print(road_endpoints)

test_basic_board()

In [ ]:
"""
GAME AGENTS
---------------------
"""

class DiceAgent:
  """
  Class: DiceAgent
  ---------------------
  DiceAgent represents the random agent responsible for the
  roll of the dice for resources each turn.  It generates a
  number from 1-12 with the correct probability distribution
  corresponding to rolling 2 6-sided dice.
  ---------------------
  """

  def __init__(self, numDiceSides = 6):
    self.agentType = AGENT[0]
    self.NUM_DICE_SIDES = numDiceSides

  def rollDice(self):
    """
    Method: rollDice
    ----------------------
    Parameters: NA
    Returns: an integer representing the result of a random
      roll of 2 6-sided dice
    ----------------------
    """
    return randint(1, self.NUM_DICE_SIDES) + randint(1, self.NUM_DICE_SIDES)

  def getRollDistribution(self):
    """
    Method: getRollDistribution
    -----------------------------
    Parameters: NA
    Returns: a list of (ROLL, PROBABILITY) tuples containing
      all the possible rolls and the probabilities that they will be rolled.
    -----------------------------
    """
    # Tally up all the possible roll combinations
    # and the number of dice roll combinations per dice total
    totalRolls = 0
    rollCounter = Counter()
    for dice1 in range(1, self.NUM_DICE_SIDES + 1):
      for dice2 in range(1, self.NUM_DICE_SIDES + 1):
        rollCounter[dice1 + dice2] += 1
        totalRolls += 1

    # Return the list of probability tuples
    return [(roll, rollCounter[roll] / float(totalRolls)) for roll in rollCounter]

  def deepCopy(self):
    """
    Method: deepCopy
    -------------------------
    Parameters: NA
    Returns: a new DiceAgent object

    Returns a copy of this agent (which is essentially just a new DiceAgent).
    -------------------------
    """
    return DiceAgent()


class PlayerAgent(object):
  """
  Class: PlayerAgent
  ---------------------
  PlayerAgent defines a generic player agent in Settlers consisting of a name,
  player index, and player stats/game-specific information like number
  of victory points, lists of all roads, settlements, and cities owned by the
  player, and a counter of resources that the player has.

  Instance Variables:
  ---
  agentType = the type of game agent (PLAYER_AGENT)
  name = a string containing the name of the player
  agentIndex = the player index
  victoryPoints = the number of victory points the player has
  roads = a list of Tiles objects representing the roads a player has
  settlements = a list of Tiles objects representing the settlements a player has
  cities = a list of Tiles objects representing the cities a player has
  resources = a Counter containing the count of each resource type (in ResourceTypes) the player has
  ---------------------
  """

  def __init__(self, name, agentIndex):
    self.agentType = AGENT[1]
    self.name = name
    self.agentIndex = agentIndex
    self.victoryPoints = 0

    # List of roads
    self.roads = []

    # List of settlements 
    self.settlements = []

    # List of Cities owned
    self.cities = []

    # Counter of resources initialized to zero
    self.resources = Counter({i: 0 for i in range(5)})
    
  def __repr__(self):
    """
    Method: __repr__
    ---------------------
    Parameters: NA
    Returns: a string representation of the current PlayerAgent.
    ---------------------
    """
    s = "---------- " + self.name + " ----------\n"
    s += "Victory points: " + str(self.victoryPoints) + "\n"
    s += "Resources: " + str(self.resources) + "\n"
    s += "Settlements (" + str(len(self.settlements)) + "): " + str(self.settlements) + "\n"
    s += "Roads (" + str(len(self.roads)) + "): " + str(self.roads) + "\n"
    s += "Cities (" + str(len(self.cities)) + "): " + str(self.cities) + "\n"
    s += "--------------------------------------------\n"
    return s


  def canSettle(self):
    """
    Method: canSettle
    ---------------------
    Parameters: NA
    Returns: True/False whether or not this PlayerAgent has enough
      resources to build a new settlement (based on the SETTLEMENT_COST constant)
    ---------------------
    """
    modifiedResources = copy.deepcopy(self.resources)
    modifiedResources.subtract(SETTLEMENT_COST)

    # If any resource counts dip below 0, we don't have enough
    for resourceType in modifiedResources:
      if modifiedResources[resourceType] < 0:
        return False

    return True

  def canBuildCity(self):
    """
    Method: canBuildCity
    ----------------------
    Parameters: NA
    Returns: True/False whether or not this PlayerAgent has enough
      resources to build a new city (based on the CITY_COST constant)
    ----------------------
    """
    modifiedResources = copy.deepcopy(self.resources)
    modifiedResources.subtract(CITY_COST)

    # If any resource counts dip below 0, we don't have enough
    for resourceType in modifiedResources:
      if modifiedResources[resourceType] < 0:
        return False

    return True

  def canBuildRoad(self):
    """
    Method: canBuildRoad
    ----------------------
    Parameters: NA
    Returns: True/False whether or not this PlayerAgent has enough
      resources to build a new road (based on the ROAD_COST constant)
    ----------------------
    """
    modifiedResources = copy.deepcopy(self.resources)
    modifiedResources.subtract(ROAD_COST)

    # If any resource counts dip below 0, we don't have enough
    for resourceType in modifiedResources:
      if modifiedResources[resourceType] < 0:
        return False

    return True

  def deepCopy(self, board):
    """
    Method: deepCopy
    ----------------------
    Parameters:
      board - the current state of the board (an instance of Board)
    Returns: a deep copy of this instance of PlayerAgent, including full
      copies of all instance Variables
    ----------------------
    """
    newCopy = PlayerAgent(self.name, self.agentIndex)
    newCopy.victoryPoints = self.victoryPoints
    newCopy.roads = [board.getTile(road.X, road.Y) for road in self.roads]
    newCopy.settlements = [board.getTile(settlement.X, settlement.Y) for settlement in self.settlements]
    newCopy.resources = copy.deepcopy(self.resources)
    newCopy.cities = [board.getTile(city.X, city.Y) for city in self.cities]
    return newCopy

  def applyAction(self, action, board):
    """
    Method: applyAction
    -----------------------
    Parameters:
      action - the action tuple (ACTION, LOCATION) to applyAction
    Returns: NA

    Applies the given action tuple to the current player.  Does this
    by deducting resources appropriately and adding to/removing from the player's
    lists of roads, settlements, and cities.
    -----------------------
    """
    if action is None:
      return

    # Settling
    if action[0] is Actions["SETTLE"]:
      if not self.canSettle():
        raise Exception("Player " + str(self.agentIndex) + " doesn't have enough resources to build a settlement!")
      
      # Add this settlement to our settlements list and update victory
      # points and resources
      actionTile = action[1]
      tile = board.getTile(actionTile.x, actionTile.y)
      board.applyAction(self.agentIndex, action)
      self.settlements.append(tile)
      self.resources.subtract(SETTLEMENT_COST)
      self.victoryPoints += SETTLEMENT_VICTORY_POINTS

    # Building a road
    if action[0] is Actions["ROAD"]:
      if not self.canBuildRoad():
        raise Exception("Player " + str(self.agentIndex) + " doesn't have enough resources to build a road!")

      # Add this road to our roads list and update resources
      actionTile = action[1]
      road = board.getTile(actionTile.x, actionTile.y)
      board.applyAction(self.agentIndex, action)
      self.roads.append(road)
      self.resources.subtract(ROAD_COST)

    # Building a city
    if action[0] is Actions["CITY"]:
      if not self.canBuildCity():
        raise Exception("Player " + str(self.agentIndex) + " doesn't have enough resources to build a city!")
      
      # Add this city to our list of cities and remove this city
      # from our list of settlements (since it was formerly a settlement)
      actionTile = action[1]
      tile = board.getTile(actionTile.x, actionTile.y)
      board.applyAction(self.agentIndex, action)
      self.cities.append(tile)
      for settlement in self.settlements:
        if settlement.x == tile.x and settlement.y == tile.y:
          self.settlements.remove(settlement)
          break

      # and update victory points and resources
      self.resources.subtract(CITY_COST)
      self.victoryPoints += CITY_VICTORY_POINTS

  def updateResources(self, diceRoll, board):
    """
    Method: updateResources
    -----------------------------
    Parameters:
      diceRoll - the sum of the two dice Rolled
      board - a Board object representing the current board state
    Returns: a Counter containing the number of each resource gained

    Takes the current dice roll and board setup, and awards
    the current player resources depending on built settlements on the board.
    Returns the count of each resource that the player gained.
    -----------------------------
    """
    newResources = board.getResourcesFromDieRoll(self.agentIndex, diceRoll)
    self.resources += newResources
    return newResources


  def collectInitialResources(self, board):
    """
    Method: collectInitialResources
    --------------------------------
    Parameters:
      board - a Board object representing the current board state

    Returns: NA

    Takes the current board setup and awards the current player
    resources for each of his/her current settlements.  For example,
    if the player had a settlement bordering BRICK and ORE and another
    one bordering BRICK, this player would receive 2 BRICK and 1 ORE.
    --------------------------------
    """
    # Get resources for each settlement
    for settlement in self.settlements:
      # Find all tiles bordering this settlement and
      # take 1 resource of each of the surrounding tile types
      tile = board.getTile(settlement.x, settlement.y)
      self.resources[tile.resource] += 1


  def hasWon(self):
    """
    Method: hasWon
    -----------------------------
    Parameters: NA
    Returns: True/False whether or not the curernt player
      has won the game (AKA met or exceeded VICTORY_POINTS_TO_WIN)
    -----------------------------
    """
    return self.victoryPoints >= VICTORY_POINTS_TO_WIN


  def getAction(self, state):
    """
    Method: getAction
    -----------------------------
    Parameters:
      state - a GameState object containing information about the current state of the game
    Returns: an action tuple (ACTION, LOCATION) of the action this player should take
    
    Note: must be overridden by a subclass
    -----------------------------
    """
    raise Exception("Cannot get action for superclass - must implement getAction in PlayerAgent subclass!")


In [ ]:
def test_PlayerAgent():
    # Create a PlayerAgent instance
    player = PlayerAgent("Player 1", 1)

    # Test the __repr__() method
    print(player)
    
    # Test the canSettle() method
    assert player.canSettle() == False

    # Test the canBuildCity() method
    assert player.canBuildCity() == False

    # Test the canBuildRoad() method
    assert player.canBuildRoad() == False

    # Test the deepCopy() method
    board = BasicBoard()  # Assuming you have a Board class
    new_player = player.deepCopy(board)
    assert new_player.name == player.name
    assert new_player.agentIndex == player.agentIndex
    assert new_player.victoryPoints == player.victoryPoints
    assert new_player.roads == player.roads
    assert new_player.settlements == player.settlements
    assert new_player.resources == player.resources
    assert new_player.cities == player.cities

    player.resources = Counter({i: 3 for i in range(5)})

    # Test the applyAction() method
    action = (Actions["SETTLE"], board.getTile(0, 0))  # Assuming you have Actions and Location defined
    player.applyAction(action, board)

    board.printBoard()

    assert player.settlements == [board.getTile(0, 0)]
    assert player.victoryPoints == SETTLEMENT_VICTORY_POINTS

    # Test the updateResources() method
    diceRoll = 8  # Assuming you have a diceRoll value
    player.updateResources(diceRoll, board)

    # Test the collectInitialResources() method
    player.settlements = [board.getTile(0, 0), board.getTile(1, 1)]  # Assuming you have a board setup
    player.collectInitialResources(board)

    print(player)

    # Test the hasWon() method
    player.victoryPoints = VICTORY_POINTS_TO_WIN
    assert player.hasWon() == True

    print("All tests passed!")

# Run the test_PlayerAgent() function
test_PlayerAgent()

In [ ]:
class GameState:
  """
  Class: GameState
  -------------------------------
  A class representing all information about the current state of the game.
  Includes a Board object representing the current state of the game board,
  as well as a list of all agents (players) in the game.
  -------------------------------
  """

  def __init__(self):
    """
    Method: __init__
    -----------------------------
    Parameters:
      prevState - an optional GameState object to pass in.  If this is passed
        in, this new GameState object will instead be cloned from prevState
      layout - an optional board layout to pass in to define the layout
        of the game board

    Returns: NA

    Initializes the GameState object, either by creating a new one from
    scratch or by cloning an optionally passed-in other GameState object.
    Can also optionally define the board layout if you are creating a new
    GameState object from scratch.
    ------------------------------
    """
    
    self.board = BasicBoard()
    self.playerAgents = [PlayerAgent("yera", 0), PlayerAgent("krati", 1), PlayerAgent("juan", 2), PlayerAgent("isi", 3)]

    # Make the dice agent
    self.diceAgent = DiceAgent()

  def deepCopy(self):
    copy = GameState()
    copy.board = self.board.deepCopy()
    copy.playerAgents = [playerAgent.deepCopy(copy.board) for playerAgent in self.playerAgents]
    return copy


  def getLegalActions(self, agentIndex):
    """
    Method: getLegalActions
    ------------------------------
    Parameters:
      agentIndex - the index of the agent to return legal actions for

    Returns: a list of action tuples (ACTION, LOCATION) (e.g. (ACTIONS.SETTLE, *some Tile object*))
      representing all the valid actions that the given agent/player can take
    ------------------------------
    """
    legalActions = set()
    if self.gameOver() >= 0: return legalActions
    agent = self.playerAgents[agentIndex]

    # If they can build a road...
    if agent.canBuildRoad():
      # Look at all unoccupied edges coming from the player's existing settlements and cities
      agentSettlements = []; agentSettlements.extend(agent.settlements); agentSettlements.extend(agent.cities)
      for settlement in agentSettlements:

        # TODO: getEdgesOfVertex is not implemented
        currEdges = self.board.getEdgesOfVertex(settlement)
        for currEdge in currEdges:
          if not currEdge.isOccupied():
            if (Actions["ROAD"], currEdge) not in legalActions:
              legalActions.add((Actions["ROAD"], currEdge))

      # Look at all unoccupied edges coming from the player's existing roads
      for road in agent.roads:
        # TODO: getVertexEnds is not implemented
        currVertices = self.board.getVertexEnds(road)
        for vertex in currVertices:
          if vertex.player != None and vertex.player != agentIndex: continue
          currEdges = self.board.getEdgesOfVertex(vertex)
          for currEdge in currEdges:
            if not currEdge.isOccupied(): 
              if (Actions["ROAD"], currEdge) not in legalActions:
                legalActions.add((Actions["ROAD"], currEdge)) 

    # If they can settle...
    if agent.canSettle():

      # Look at all unoccupied endpoints of the player's existing roads
      for road in agent.roads:
        possibleSettlements = self.board.getVertexEnds(road)
        for possibleSettlement in possibleSettlements:
          if possibleSettlement.canSettle:
            if (Actions["SETTLE"], possibleSettlement) not in legalActions:
              legalActions.add((Actions["SETTLE"], possibleSettlement))

    # If they can build a city...
    if agent.canBuildCity():
      # All current settlements are valid city locations
      for settlement in agent.settlements:
        legalActions.add((Actions["CITY"], settlement))
    return list(legalActions)

  def generateSuccessor(self, playerIndex, action):
    """
    Method: generateSuccessor
    ----------------------------
    Parameters:
      playerIndex - the number of the player that is about to take an action
      action - the action that the player is about to take

    Returns: a new GameState object with playerIndex having taken 'action'

    Creates a clone of the current game state, and then performs the
    given action on behalf of the given player.  Returns the resulting
    GameState object.
    ----------------------------
    """
    if self.gameOver() >= 0:
      raise Exception("Can\'t generate a successor of a terminal state!")

    # Create a copy of the current state, and perform the given action
    # for the given player
    copy = self.deepCopy()
    copy.playerAgents[playerIndex].applyAction(action, copy.board)
    copy.board.applyAction(playerIndex, action)
    return copy

  def makeMove(self, playerIndex, action):
    """
    Method: makeMove
    ----------------------------
    Parameters:
      playerIndex - the number of the player that is about to take an action
      action - the action that the player is about to take

    Modifies the current game state to reflect the action that is passed in
    ----------------------------
    """
    self.playerAgents[playerIndex].applyAction(action, self.board)
    self.board.applyAction(playerIndex, action)

  def getNumPlayerAgents(self):
    """
    Method: getNumPlayerAgents
    ----------------------------
    Parameters: NA
    Returns: the number of PLAYER agents (players) in the game
    ----------------------------
    """
    return len(self.playerAgents)

  def gameOver(self):
    """
    Method: gameOver
    ----------------------------
    Parameters: NA
    Returns: the index of the player that has won, or -1 if the game has not ended
    ----------------------------
    """
    # See if any of the agents have won
    for agent in self.playerAgents:
      if agent.hasWon():
        return agent.agentIndex
    return -1

  def updatePlayerResourcesForDiceRoll(self, diceRoll):
    """
    Method: updatePlayerResourcesForDiceRoll
    -----------------------------------------
    Parameters:
      diceRoll - the dice total of the 2 rolled 6-sided dice
        to use to distribute more resources
    Returns: NA

    Updates the resource counts of all agents based on the
    given dice roll.
    -----------------------------------------
    """
    for agent in self.playerAgents:
      gainedResources = agent.updateResources(diceRoll, self.board)
      if VERBOSE:
        print(str(agent.name) + " received: " + str(gainedResources))
        print(str(agent.name) + " now has: " + str(agent.resources))


class Game:
  """
  Class: Game
  ------------------------
  Represents all information about a game, and controls game flow.
  In addition to containing a GameState object to keep track of all game
  state, a Game object also contains the game's move history as a list
  of (AGENTNAME, ACTION) tuples.
  ------------------------
  """

  def __init__(self, playerAgentNums = None):
    """
    Method: __init__
    ----------------------
    Parameters:
      gameState - an optional pre-defined GameState object to use for the game.
        If one isn't passed in, the Game begins with a newly-created GameState object.

    Returns: NA

    Initializes the Game object by initializing the move history list
    and the internal GameState object.
    ----------------------
    """
    self.moveHistory = []
    self.gameState = GameState()
    self.playerAgentNums = playerAgentNums 

  def start(self):
    """
    Method: start
    ----------------------
    Parameters: NA
    Returns: NA

    Begins the game by running the main game loop.
    ----------------------
    """
    # Welcome message
    if VERBOSE:
      print("WELCOME TO SETTLERS OF CATAN!")
      print("-----------------------------")

    # Turn tracking
    turnNumber = 1
    currentAgentIndex = 0

    # Each player has to place 2 settlements and 2 roads
    for i in range(2):
      for agentIndex in range(self.gameState.getNumPlayerAgents()):
        currentAgent = self.gameState.playerAgents[agentIndex]
        legalActions = self.gameState.getLegalActions(agentIndex)

        if VERBOSE:
          print("It's " + str(currentAgent.name) + "'s turn!. Where do you want to place your settlement?")
          self.gameState.board.printBoard()
       
        x = input()
        action = (Actions["SETTLE"], self.gameState.board.getTile(x[0], x[1]))
    
        self.gameState.board.applyAction(agentIndex, action)
        self.moveHistory.append((currentAgent.name, action))

        if VERBOSE:
          print("It's " + str(currentAgent.name) + "'s turn!. Where do you want to place your road?")
          self.gameState.board.printBoard()
        
        x = input()
        action = (Actions["ROAD"], self.gameState.board.getTile(x[0], x[1]))

        self.gameState.board.applyAction(agentIndex, action)
        self.moveHistory.append((currentAgent.name, action))

        currentAgentIndex = (currentAgentIndex+1) % self.gameState.getNumPlayerAgents()
        turnNumber += 1

    while (self.gameState.gameOver() < 0):
      # Initial information
      currentAgent = self.gameState.playerAgents[currentAgentIndex]
      if VERBOSE:
        print("---------- TURN " + str(turnNumber) + " --------------")
        print("It's " + str(currentAgent.name) + "'s turn!")

      # Print player info
      if VERBOSE:
        print("PLAYER INFO:")
        for a in self.gameState.playerAgents:
          print(a)

      # Dice roll + resource distribution
      diceRoll = self.gameState.diceAgent.rollDice()
      if VERBOSE: print("Rolled a " + str(diceRoll))
      self.gameState.updatePlayerResourcesForDiceRoll(diceRoll)
      # The current player performs 1 action, input the action from the list of legal actions
      action = currentAgent.getAction(self.gameState)
      if VERBOSE: 
        print("Best Action: " + str(action))
      currentAgent.applyAction(action, self.gameState.board)
      self.gameState.board.applyAction(currentAgent.agentIndex, action)
      
      if VERBOSE:# Print out the updated game state
        if (action != None):
          print(str(currentAgent.name) + " took action " + str(action[0]) + " at " + str(action[1]) + "\n")
        else:
          print(str(currentAgent.name) + " had no actions to take")
      # Track the game's move history
      self.moveHistory.append((currentAgent.name, action))
      # Go to the next player/turn
      currentAgentIndex = (currentAgentIndex+1) % self.gameState.getNumPlayerAgents()
      turnNumber += 1

      # Caps the total number of iterations for a game
      if turnNumber > CUTOFF_TURNS: break

    winner = self.gameState.gameOver()
    if winner < 0: return (winner, turnNumber, -1)
    agentWinner = self.gameState.playerAgents[winner]
    agentLoser = self.gameState.playerAgents[1-winner]
    if VERBOSE: print(agentWinner.name + " won the game")
    return (winner, turnNumber, agentWinner.victoryPoints - agentLoser.victoryPoints)
    


  def run(self):    
    """
    Method: run
    ----------------------
    Parameters: NA
    Returns: NA

    Runs the main game loop.  Initializes all the players' resources
    and settlements, prints out the game state, and then begins the main loop.
    Each turn, the dice are rolled, and all players get resources.  Then, the
    player whose turn it is can take 1 action, and the game state is updated
    accordingly.  The game continues until 1 player wins by reaching
    VICTORY_POINTS_TO_WIN victory points.
    ----------------------
    """
    # Welcome message
    if VERBOSE:
      print("WELCOME TO SETTLERS OF CATAN!")
      print("-----------------------------")
  
    # Turn tracking
    turnNumber = 1
    currentAgentIndex = 0
    # Main game loop
    while (self.gameState.gameOver() < 0):
      # Initial information
      currentAgent = self.gameState.playerAgents[currentAgentIndex]
      if VERBOSE:
        print("---------- TURN " + str(turnNumber) + " --------------")
        print("It's " + str(currentAgent.name) + "'s turn!")

      # Print player info
      if VERBOSE:
        print("PLAYER INFO:")
        for a in self.gameState.playerAgents:
          print(a)

      # Dice roll + resource distribution
      diceRoll = self.gameState.diceAgent.rollDice()
      if VERBOSE: print("Rolled a " + str(diceRoll))
      self.gameState.updatePlayerResourcesForDiceRoll(diceRoll)
      # The current player performs 1 action
      value, action = currentAgent.getAction(self.gameState)
      if VERBOSE: 
        print("Best Action: " + str(action))
        print("Best Value: " + str(value))
      currentAgent.applyAction(action, self.gameState.board)
      self.gameState.board.applyAction(currentAgent.agentIndex, action)
      
      if VERBOSE:# Print out the updated game state
        if (action != None):
          print(str(currentAgent.name) + " took action " + str(action[0]) + " at " + str(action[1]) + "\n")
        else:
          print(str(currentAgent.name) + " had no actions to take")
      # Track the game's move history
      self.moveHistory.append((currentAgent.name, action))
      # Go to the next player/turn
      currentAgentIndex = (currentAgentIndex+1) % self.gameState.getNumPlayerAgents()
      turnNumber += 1

      # Caps the total number of iterations for a game
      if turnNumber > CUTOFF_TURNS: break

    winner = self.gameState.gameOver()
    if winner < 0: return (winner, turnNumber, -1)
    agentWinner = self.gameState.playerAgents[winner]
    agentLoser = self.gameState.playerAgents[1-winner]
    if VERBOSE: print(agentWinner.name + " won the game")
    return (winner, turnNumber, agentWinner.victoryPoints - agentLoser.victoryPoints)


In [ ]:
Game().start()